PIGS ANALYSIS NOTEBOOK  
Patrick M. Donnelly  
University of Washington  
August 6th, 2018  

In [1]:
# import necessary databases and libraries
import pycurl,json,requests,sys,os
import pandas as pd
import numpy as np
# existing redcap API stipulates StringIO, but pycurl now works with BytesIO
from io import BytesIO

In [95]:
# get api token
# if you need an access token, consult the RedCap API page
home = os.path.expanduser('~')
with open (home+"/Desktop/redcap_apitoken_repo.txt", "r") as myfile:
    token=myfile.read().split()

In [123]:
# say which report you want
# report 27719 is the report in the screening database called "pigs_data"
report = {
    'token': token,
    'content': 'report',
    'format': 'csv',
    'report_id': '27719',
    'rawOrLabel': 'raw',
    'rawOrLabelHeaders': 'raw',
    'exportCheckboxLabel': 'false',
    'returnFormat': 'csv'
}

In [124]:
# specify API address
redcap_path = 'https://redcap.iths.org/api/'
# use requests module to pull data from database using api token
# and convert to csv
pull_pigs = requests.post(redcap_path, data=report)
report_filename =os.path.join(home+'/Downloads/pigs_data.csv')
with open(report_filename, 'w') as report_file:
        report_file.write(pull_pigs.text)
pigs_data = pd.read_csv(report_filename)
visit1 = pigs_data[pigs_data['study_name'] == 41]
visit2 = pigs_data[pigs_data['study_name'] == 42]

In [125]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
from scipy.stats import linregress

In [126]:
for ii in range(0, len(pigs_data['record_id'])):
    if pigs_data['study_name'][ii] == 41:
        if pigs_data['pigs_group'][ii] == 0 or 2:
            pigs_data.loc[ii,'pigs_p1_r1_acc'] = pigs_data.loc[ii,'pigs_p1_r1_acc']/237
            pigs_data.loc[ii,'pigs_p1_r2_acc'] = pigs_data.loc[ii,'pigs_p1_r2_acc']/237
            pigs_data.loc[ii,'pigs_p2_r1_acc'] = pigs_data.loc[ii,'pigs_p2_r1_acc']/223
            pigs_data.loc[ii,'pigs_p2_r2_acc'] = pigs_data.loc[ii,'pigs_p2_r2_acc']/223
            pigs_data.loc[ii,'pigs_p3_r1_acc'] = pigs_data.loc[ii,'pigs_p3_r1_acc']/301
            pigs_data.loc[ii,'pigs_p3_r2_acc'] = pigs_data.loc[ii,'pigs_p3_r2_acc']/301
            pigs_data.loc[ii,'pigs_p4_r1_acc'] = pigs_data.loc[ii,'pigs_p4_r1_acc']/300
            pigs_data.loc[ii,'pigs_p4_r2_acc'] = pigs_data.loc[ii,'pigs_p4_r2_acc']/300
        elif pigs_data['pigs_group'][ii] == 1 or 3:
            pigs_data.loc[ii,'pigs_p1_r1_acc'] = pigs_data.loc[ii,'pigs_p1_r1_acc']/301
            pigs_data.loc[ii,'pigs_p1_r2_acc'] = pigs_data.loc[ii,'pigs_p1_r2_acc']/301
            pigs_data.loc[ii,'pigs_p2_r1_acc'] = pigs_data.loc[ii,'pigs_p2_r1_acc']/300
            pigs_data.loc[ii,'pigs_p2_r2_acc'] = pigs_data.loc[ii,'pigs_p2_r2_acc']/300
            pigs_data.loc[ii,'pigs_p3_r1_acc'] = pigs_data.loc[ii,'pigs_p3_r1_acc']/237
            pigs_data.loc[ii,'pigs_p3_r2_acc'] = pigs_data.loc[ii,'pigs_p3_r2_acc']/237
            pigs_data.loc[ii,'pigs_p4_r1_acc'] = pigs_data.loc[ii,'pigs_p4_r1_acc']/223
            pigs_data.loc[ii,'pigs_p4_r2_acc'] = pigs_data.loc[ii,'pigs_p4_r2_acc']/223
        
    elif pigs_data['study_name'][ii] == 42:
        if pigs_data['pigs_group'][ii] == 0 or 2:
            pigs_data.loc[ii,'pigs_p1_r1_acc'] = pigs_data.loc[ii,'pigs_p1_r1_acc']/259
            pigs_data.loc[ii,'pigs_p1_r2_acc'] = pigs_data.loc[ii,'pigs_p1_r2_acc']/259
            pigs_data.loc[ii,'pigs_p2_r1_acc'] = pigs_data.loc[ii,'pigs_p2_r1_acc']/258
            pigs_data.loc[ii,'pigs_p2_r2_acc'] = pigs_data.loc[ii,'pigs_p2_r2_acc']/258
            pigs_data.loc[ii,'pigs_p3_r1_acc'] = pigs_data.loc[ii,'pigs_p3_r1_acc']/359
            pigs_data.loc[ii,'pigs_p3_r2_acc'] = pigs_data.loc[ii,'pigs_p3_r2_acc']/359
            pigs_data.loc[ii,'pigs_p4_r1_acc'] = pigs_data.loc[ii,'pigs_p4_r1_acc']/333
            pigs_data.loc[ii,'pigs_p4_r2_acc'] = pigs_data.loc[ii,'pigs_p4_r2_acc']/333          
        elif pigs_data['pigs_group'][ii] == 1 or 3:
            pigs_data.loc[ii,'pigs_p1_r1_acc'] = pigs_data.loc[ii,'pigs_p1_r1_acc']/359
            pigs_data.loc[ii,'pigs_p1_r2_acc'] = pigs_data.loc[ii,'pigs_p1_r2_acc']/359
            pigs_data.loc[ii,'pigs_p2_r1_acc'] = pigs_data.loc[ii,'pigs_p2_r1_acc']/333
            pigs_data.loc[ii,'pigs_p2_r2_acc'] = pigs_data.loc[ii,'pigs_p2_r2_acc']/333
            pigs_data.loc[ii,'pigs_p3_r1_acc'] = pigs_data.loc[ii,'pigs_p3_r1_acc']/259
            pigs_data.loc[ii,'pigs_p3_r2_acc'] = pigs_data.loc[ii,'pigs_p3_r2_acc']/259
            pigs_data.loc[ii,'pigs_p4_r1_acc'] = pigs_data.loc[ii,'pigs_p4_r1_acc']/258
            pigs_data.loc[ii,'pigs_p4_r2_acc'] = pigs_data.loc[ii,'pigs_p4_r2_acc']/258

KeyboardInterrupt: 

In [117]:
ii=0
pigs_data.loc[ii,'pigs_p1_r1_acc']
pigs_data['pigs_group'][ii] == (0 or 2)
#pigs_data.loc[ii,'pigs_group']
#pigs_data['study_name'][ii]
#pigs_data.loc[ii,'pigs_p1_r1_acc']
#pigs_data.loc[ii,'pigs_p1_r1_acc']/237
#pigs_data.loc[ii,'pigs_p1_r1_acc'] = pigs_data.loc[ii,'pigs_p1_r1_acc']/237.0

False

In [29]:
pigs_data.loc[ii,'pigs_p1_r1_acc']

0.91561181434599159

In [ ]:
pigs_data['pigs_p1_r1_acc']

In [23]:
pigs_data

,record_id,redcap_event_name,study_name,visit_researcher,visit_dt,int_session,int_time,int_days,int_hours,int_period,...,pigs_word2_time,pigs_word2_acc,pigs_word2_rate,pigs_pseudo1_time,pigs_pseudo1_acc,pigs_pseudo1_rate,pigs_pseudo2_time,pigs_pseudo2_acc,pigs_pseudo2_rate,scores_complete
0,274,visit_4_arm_1,41,TM,2018-07-18 10:00,1,0.0,0.0,0.0,NaN,...,83.0,17,0.205,101.0,11,0.109,86.0,11,0.128,2
1,274,visit_5_arm_1,42,TM,2018-08-02 10:00,2,NaN,NaN,NaN,NaN,...,95.0,25,0.263,75.0,16,0.213,78.0,19,0.244,2
2,280,visit_2_arm_1,41,TM,2018-07-26 15:40,0,0.0,0.0,0.0,NaN,...,140.0,21,0.150,319.0,12,0.038,279.0,19,0.068,2
3,280,visit_3_arm_1,42,PD,2018-08-10 15:43,2,NaN,NaN,NaN,NaN,...,NaN,20,NaN,NaN,17,NaN,NaN,15,NaN,2
4,486,visit_6_arm_1,41,TM,2018-07-23 10:00,1,0.0,0.0,0.0,NaN,...,40.0,25,0.625,80.0,16,0.200,74.0,20,0.270,2
5,486,visit_7_arm_1,42,TM,2018-08-07 09:00,2,NaN,NaN,NaN,NaN,...,38.0,22,0.579,96.0,13,0.135,67.0,14,0.209,2
6,561,visit_4_arm_1,41,PD,2018-07-18 13:30,1,0.0,0.0,0.0,NaN,...,28.6,29,1.010,53.0,25,0.470,58.0,19,0.330,2
7,561,visit_5_arm_1,42,PD,2018-08-02 13:00,2,NaN,NaN,NaN,NaN,...,30.0,29,0.967,66.0,22,0.333,65.0,22,3.667,2
8,611,visit_2_arm_1,41,PD,2018-07-02 14:00,1,0.0,0.0,0.0,NaN,...,55.0,17,0.309,135.0,9,0.067,138.0,13,0.094,2
9,611,visit_3_arm_1,42,PD,2018-07-16 15:00,2,NaN,NaN,NaN,NaN,...,93.0,17,0.183,130.0,12,0.092,142.0,15,0.106,2


In [ ]:
pigs_data['pigs_group'][ii]

In [ ]:
pigs_data['short_first_time'] = np.nan
pigs_data['short_first_acc']  = np.nan
pigs_data['short_first_rate']  = np.nan
pigs_data['long_first_time']  = np.nan
pigs_data['long_first_acc']  = np.nan
pigs_data['long_first_rate']  = np.nan
pigs_data['short_second_time']  = np.nan
pigs_data['short_second_acc'] = np.nan
pigs_data['short_second_rate'] = np.nan
pigs_data['long_second_time'] = np.nan
pigs_data['long_second_acc'] = np.nan
pigs_data['long_second_rate'] = np.nan

for ii in range(0, len(pigs_data['record_id'])):
#    if pigs_data['pigs_group'][ii] == 'A1' or 'B1':
        pigs_data.loc[ii,'short_first_time'] = np.mean([pigs_data['pigs_p1_r1_time'][ii], pigs_data['pigs_p2_r1_time'][ii]])
        pigs_data.loc[ii,'short_first_acc'] = np.mean([pigs_data['pigs_p1_r1_acc'][ii], pigs_data['pigs_p2_r1_acc'][ii]])
        pigs_data.loc[ii,'short_first_rate'] = np.mean([pigs_data['pigs_p1_r1_rate'][ii], pigs_data['pigs_p2_r1_rate'][ii]])
        pigs_data.loc[ii,'long_first_time'] = np.mean([pigs_data['pigs_p3_r1_time'][ii], pigs_data['pigs_p4_r1_time'][ii]])
        pigs_data.loc[ii,'long_first_acc'] = np.mean([pigs_data['pigs_p3_r1_acc'][ii], pigs_data['pigs_p4_r1_acc'][ii]])
        pigs_data.loc[ii,'long_first_rate'] = np.mean([pigs_data['pigs_p3_r1_rate'][ii], pigs_data['pigs_p4_r1_rate'][ii]])
        
        pigs_data.loc[ii,'short_second_time'] = np.mean([pigs_data['pigs_p1_r2_time'][ii], pigs_data['pigs_p2_r2_time'][ii]])
        pigs_data.loc[ii,'short_second_acc'] = np.mean([pigs_data['pigs_p1_r2_acc'][ii], pigs_data['pigs_p2_r2_acc'][ii]])
        pigs_data.loc[ii,'short_second_rate'] = np.mean([pigs_data['pigs_p1_r2_rate'][ii], pigs_data['pigs_p2_r2_rate'][ii]])
        pigs_data.loc[ii,'long_second_time'] = np.mean([pigs_data['pigs_p3_r2_time'][ii], pigs_data['pigs_p4_r2_time'][ii]])
        pigs_data.loc[ii,'long_second_acc'] = np.mean([pigs_data['pigs_p3_r2_acc'][ii], pigs_data['pigs_p4_r2_acc'][ii]])
        pigs_data.loc[ii,'long_second_rate'] = np.mean([pigs_data['pigs_p3_r2_rate'][ii], pigs_data['pigs_p4_r2_rate'][ii]])


In [ ]:
pigs_data['word_time'] = np.nan
pigs_data['word_acc'] = np.nan
pigs_data['word_rate'] = np.nan
pigs_data['pseudo_time'] = np.nan
pigs_data['pseudo_acc'] = np.nan
pigs_data['pseudo_rate'] = np.nan

for ii in range(0, len(pigs_data['record_id'])):
    pigs_data.loc[ii, 'word_time'] = np.mean([pigs_data['pigs_word1_time'][ii], pigs_data['pigs_word2_time'][ii]])
    pigs_data.loc[ii, 'word_acc'] = np.mean([pigs_data['pigs_word1_acc'][ii], pigs_data['pigs_word2_acc'][ii]])
    pigs_data.loc[ii, 'word_rate'] = np.mean([pigs_data['pigs_word1_rate'][ii], pigs_data['pigs_word2_rate'][ii]])
    pigs_data.loc[ii, 'pseudo_time'] = np.mean([pigs_data['pigs_pseudo1_time'][ii], pigs_data['pigs_pseudo2_time'][ii]])
    pigs_data.loc[ii, 'pseudo_acc'] = np.mean([pigs_data['pigs_pseudo1_acc'][ii], pigs_data['pigs_pseudo2_acc'][ii]])
    pigs_data.loc[ii, 'pseudo_rate'] = np.mean([pigs_data['pigs_pseudo1_rate'][ii], pigs_data['pigs_pseudo2_rate'][ii]])

for jj in range(1, len(pigs_data['record_id']), 2):
    pigs_data.loc[jj, 'word_acc_diff'] = pigs_data['word_acc'][pigs_data['study_name'] == 42][jj] - pigs_data['word_acc'][pigs_data['study_name'] == 41][jj-1]
    pigs_data.loc[jj, 'pseudo_acc_diff'] = pigs_data['pseudo_acc'][pigs_data['study_name'] == 42][jj] - pigs_data['pseudo_acc'][pigs_data['study_name'] == 41][jj-1]

In [ ]:
passage_data = pigs_data[['pigs_casecontrol', 'study_name', 'short_first_time', 
                          'short_first_acc', 'short_first_rate', 'long_first_time', 
                          'long_first_acc','long_first_rate', 'short_second_time', 
                          'short_second_acc', 'short_second_rate', 
                          'long_second_time', 'long_second_acc', 'long_second_rate']]
wordlist_data = pigs_data[['pigs_casecontrol', 'study_name','word_time', 'word_acc', 'word_rate',
                           'pseudo_time', 'pseudo_acc', 'pseudo_rate']]
passage_accuracy = pigs_data[['pigs_casecontrol', 'study_name','short_first_acc',
                          'long_first_acc','short_second_acc', 'long_second_acc']]
passage_rate = pigs_data[['pigs_casecontrol', 'study_name', 'short_first_rate', 
                          'long_first_rate', 'short_second_rate', 'long_second_rate']]
wordlist_acc_data = pigs_data[['pigs_casecontrol', 'study_name','word_acc','pseudo_acc']]
wordlist_acc_diff_data = pigs_data[['pigs_casecontrol', 'word_acc_diff', 'pseudo_acc_diff']]
wordlist_rate_data = pigs_data[['pigs_casecontrol', 'study_name', 'word_rate','pseudo_rate']]

In [ ]:
acc_grouped = passage_accuracy.groupby(['pigs_casecontrol', 'study_name']).mean()

In [ ]:
acc_grouped.plot(kind='bar',style='seaborn-whitegrid')

In [ ]:
rate_grouped = passage_rate.groupby(['pigs_casecontrol', 'study_name']).mean()

In [ ]:
rate_grouped.plot(kind='bar')

In [ ]:
wordlist_acc_grouped = wordlist_acc_data.groupby(['pigs_casecontrol', 'study_name']).mean()
wl_acc_grpd_error = wordlist_acc_data.groupby(['pigs_casecontrol', 'study_name']).sem()
wordlist_acc_grouped.plot(kind='bar', yerr=wl_acc_grpd_error)

In [ ]:
wordlist_acc_diff_grouped = wordlist_acc_diff_data.groupby(['pigs_casecontrol']).mean()
wl_acc_diff_grpd_error = wordlist_acc_diff_data.groupby(['pigs_casecontrol']).sem()
wordlist_acc_diff_grouped.plot(kind='bar', yerr=wl_acc_diff_grpd_error)

In [ ]:
wordlist_acc_diff_data[['word_acc_diff', 'pseudo_acc_diff']].plt.bar()

In [ ]:
control_acc = wordlist_acc_diff_data[wordlist_acc_diff_data['pigs_casecontrol']==0]
pigs_acc = wordlist_acc_diff_data[wordlist_acc_diff_data['pigs_casecontrol']==1]
#plt.scatter(wordlist_acc_diff_data['pigs_casecontrol'][wordlist_acc_diff_data['pigs_casecontrol']==0],wordlist_acc_diff_data['word_acc_diff'][wordlist_acc_diff_data['pigs_casecontrol']==0])
#plt.scatter(wordlist_acc_diff_data['pigs_casecontrol'][wordlist_acc_diff_data['pigs_casecontrol']==1],wordlist_acc_diff_data['pseudo_acc_diff'][wordlist_acc_diff_data['pigs_casecontrol']==1])
plt.plot(control_acc['pigs_casecontrol'], [control_acc[['word_acc_diff','pseudo_acc_diff']]])
plt.plot(pigs_acc['pigs_casecontrol'], [pigs_acc[['word_acc_diff','pseudo_acc_diff']]])

In [ ]:
wordlist_acc_diff_data['pigs_casecontrol'].unique()

In [ ]:
wordlist_rate_grouped = wordlist_rate_data.groupby(['pigs_casecontrol', 'study_name']).mean()

In [ ]:
wordlist_rate_grouped.plot(kind='bar')

In [ ]:
pigs_data